<a href="https://colab.research.google.com/github/lucascerfig/PyTorch-experiments/blob/master/NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
  # A classe Net sendo criada está herdando as propriedade de nn.Module
  def __init__(self): # Construtor

    super(Net, self).__init__() 

    # Conv2d(Canais de Entrada, Canais de Saída (Kernels), Tamanho dos Kernels)
    self.conv1 = nn.Conv2d(1, 6, 3) # Camada 1 (Entrada?)
    self.conv2 = nn.Conv2d(6, 16, 3) # Camada 2 

    # Operação linear (y = Wx + b)
    # Linear(Tamanho da Entrada, Tamanho da Saída)
    self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6x6 -> Dimensão da imagem
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  # Definição do Forward Pass
  def forward(self, x): # A entrada x é um tensor
    # Max Pooling numa janela 2x2
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2) # Se a entrada é quadrada, apenas um valor pode ser especificado
    x = x.view(-1, self.num_flat_features(x)) # Extrai-se a matriz de features
    x = F.relu(self.fc1(x)) # Operação Linear -> Operação Não-Linear (ReLU)
    x = F.relu(self.fc2(x))
    x = self.fc3(x) # A saída é um vetor com 10 valores
    return x

  def num_flat_features(self, x):
    size = x.size()[1:] # Retorna as dimensões exceto o valor de batch
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

In [3]:
net = Net()
print(net)

# Basta definir a função forward, que a função backward já é automaticamente definida a partir do autograd

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Matriz de pesos dos kernels da primeira convolução

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0435, -0.0710,  0.0878, -0.0065, -0.0012, -0.0818, -0.1535, -0.0436,
         -0.1194,  0.0507]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad() # Zerando o gradiente da rede
out.backward(torch.randn(1, 10)) # Efetuando o backpropagation com gradientes aleatórios 

In [0]:
# torch.Tensor - Vetor multidimentsionar que suporta operações do autograd como backward() e guarda o gradiente em relacao ao tensor.

# nn.Module - Modúlo de redes neurais. Método conveniente de encapsular parâmetros, 
#             com métodos auxiliares para utilização da GPU, exportação, carregamento, etc.

# nn.Parameter - Um tipo de Tensor, é automaticamente registrado como um parametro ao ser designado como um atributo de um Module.

# autograd.Function - Define forward e backward nas operaçõos do autograd. Cada operação de um tensor cria pelo menos um nó de Function
#                     que se conecta a funções que criaram o tensor e codifica seu histórico

In [8]:
output = net(input)
target = torch.randn(10) # Valores alvo aleatórios
target = target.view(1, -1) # Transformar na mesma dimensão da saída
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0012, grad_fn=<MseLossBackward>)


In [9]:
# Seguindo o calculo de loss
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [10]:
net.zero_grad() # Zera os gradientes de todos os parâmetros para não acumula-los aos novos gradientes

print('conv1.bias.grad before backprop')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backprop')
print(net.conv1.bias.grad)

conv1.bias.grad before backprop
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backprop
tensor([ 0.0237, -0.0149,  0.0257, -0.0082,  0.0101, -0.0074])


In [0]:
# Montando um simples SGD
learning_rate = 0.01
for f in net.parameters(): 
  f.data.sub_(f.grad.data * learning_rate) # Aplicando a cada parametro da rede

In [0]:
import torch.optim as optim

# Criando um optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [19]:
# Loop de Treinamento englobando as funções definidas
for i in range(100):
  optimizer.zero_grad()
  output = net(input)
  loss = criterion(output, target)
  print("Epoch ", i + 1, " loss: ", loss)
  loss.backward()
  optimizer.step() # Atualiza os pesos

Epoch  1  loss:  tensor(0.8795, grad_fn=<MseLossBackward>)
Epoch  2  loss:  tensor(0.8712, grad_fn=<MseLossBackward>)
Epoch  3  loss:  tensor(0.8629, grad_fn=<MseLossBackward>)
Epoch  4  loss:  tensor(0.8549, grad_fn=<MseLossBackward>)
Epoch  5  loss:  tensor(0.8469, grad_fn=<MseLossBackward>)
Epoch  6  loss:  tensor(0.8390, grad_fn=<MseLossBackward>)
Epoch  7  loss:  tensor(0.8308, grad_fn=<MseLossBackward>)
Epoch  8  loss:  tensor(0.8228, grad_fn=<MseLossBackward>)
Epoch  9  loss:  tensor(0.8143, grad_fn=<MseLossBackward>)
Epoch  10  loss:  tensor(0.8057, grad_fn=<MseLossBackward>)
Epoch  11  loss:  tensor(0.7972, grad_fn=<MseLossBackward>)
Epoch  12  loss:  tensor(0.7884, grad_fn=<MseLossBackward>)
Epoch  13  loss:  tensor(0.7793, grad_fn=<MseLossBackward>)
Epoch  14  loss:  tensor(0.7699, grad_fn=<MseLossBackward>)
Epoch  15  loss:  tensor(0.7605, grad_fn=<MseLossBackward>)
Epoch  16  loss:  tensor(0.7507, grad_fn=<MseLossBackward>)
Epoch  17  loss:  tensor(0.7407, grad_fn=<MseLoss